In [1]:
import networkx as nx
import matplotlib as plt
import numpy as np
import matplotlib.pyplot as pyt

Creamos los graficos que necesitemos

In [2]:
G1 = nx.erdos_renyi_graph(10, 0.4, seed=1) # 10 nodos, probabilidad de 0.4

G2 = nx.barabasi_albert_graph(10, 2, seed=1)

G3 = nx.watts_strogatz_graph(10, 2, 0.2, seed=1)

G4 = nx.random_tree(10, seed=1)

G5 = nx.stochastic_block_model([20, 20], [[1, 0.2], [0.2, 1]], seed=1)


In [3]:
def random_walk(G, start_node,destination_node):
    current_node = start_node
    path = [current_node]
    while current_node != destination_node:
        neighbors = list(G.neighbors(current_node))
        current_node = np.random.choice(neighbors)
        path.append(current_node)
    return path

# Now we are going to create the function hitting time, that calculates the number of steps it takes to go from one node to another using random walks

def hitting_time(G, start_node, destination_node, num_walks=1000):
    hitting_times = []
    # From node_i to node_j
    for _ in range(num_walks):
        path = random_walk(G, start_node, destination_node)
        hitting_times.append(len(path))
    # From node_j to node_i
    for _ in range(num_walks):
        path = random_walk(G, destination_node, start_node)
        hitting_times.append(len(path))
    return np.mean(hitting_times)

In [5]:
def hit(G):
    M = np.zeros((len(G.nodes), len(G.nodes)))
    for i in range(len(G.nodes)):
        for j in range(i+1, len(G.nodes)):
            M[i, j] = hitting_time(G, i, j)
            M[j, i] = M[i, j]
    return M



def imprimirColores(M):
    pyt.imshow(M)
    pyt.colorbar()
    pyt.title('Matriz de Hitting Time')
    pyt.show()
    
print(hit(G1))














[[ 0.     10.478  13.897  17.2795 10.4135 16.362  12.7395 14.156  34.982
  10.0255]
 [10.478   0.      9.6625 15.233  10.2035 14.2295  9.1095 12.76   31.844
   8.8   ]
 [13.897   9.6625  0.     14.418  10.13   11.246   8.605  12.2695 32.4155
   9.298 ]
 [17.2795 15.233  14.418   0.      9.5785 16.103   9.8685 10.0575 29.667
  12.9595]
 [10.4135 10.2035 10.13    9.5785  0.     10.3775  7.3205  8.335  28.8765
   8.406 ]
 [16.362  14.2295 11.246  16.103  10.3775  0.      9.851  13.813  34.4955
  12.958 ]
 [12.7395  9.1095  8.605   9.8685  7.3205  9.851   0.      8.488  28.4595
   8.8285]
 [14.156  12.76   12.2695 10.0575  8.335  13.813   8.488   0.     20.753
   9.627 ]
 [34.982  31.844  32.4155 29.667  28.8765 34.4955 28.4595 20.753   0.
  30.504 ]
 [10.0255  8.8     9.298  12.9595  8.406  12.958   8.8285  9.627  30.504
   0.    ]]
